# CMIP6 Cluster Regions

### Import Packages

In [ ]:
# ========== Packages ==========
import xarray as xr
import pandas as pd
import numpy as np
import os
import copy
import seaborn as sns
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import dask
import matplotlib.cm
from matplotlib import rcParams
import math
import multiprocessing as mp
from cftime import DatetimeNoLeap
import glob

%matplotlib inline

rcParams["mathtext.default"] = 'regular'

### Functions

#### Open files

In [ ]:
# ========= Helper function to open the dataset ========
def open_dataset(filename):
    ds = xr.open_dataset(filename)
    return ds

In [ ]:
# Define a helper function to open and merge datasets
def open_and_merge_datasets(folder, model, experiment_id, variables):
    filepaths = []
    for var in variables:
        path = f'../../data/CMIP6/{experiment_id}/{folder}/{var}'
        fp = glob.glob(os.path.join(path, f'CMIP.{model}.{experiment_id}.{var}_regridded.nc'))
        if fp:
            filepaths.append(fp[0])
        else:
            #print(f"No file found for variable '{var}' in model '{model}'.")
            print(fp)

    datasets = [xr.open_dataset(fp) for fp in filepaths]
    ds = xr.merge(datasets)
    return ds

#### Helper functions

In [ ]:
def drop_var(ds_dict, var):
    for name, ds in ds_dict.items():
        ds_dict[name] = ds.drop(var)
        
    return ds_dict

In [ ]:
def select_period(ds_dict, start_year=None, end_year=None, period=None, yearly_sum=False):
    '''
    Helper function to select periods and optionally compute yearly sums.
    
    Parameters:
    ds_dict (dict): Dictionary with xarray datasets.
    start_year (int): The start year of the period.
    end_year (int): The end year of the period.
    period (int, list, str, None): Single month (int), list of months (list), multiple seasons (str) to select,
                                   or None to not select any specific period.
    yearly_sum (bool): If True, compute the yearly sum over the selected period.
    '''
    
    # Create a deep copy of the original ds_dict to avoid modifying it directly
    ds_dict_copy = copy.deepcopy(ds_dict)

    # Define season to month mapping for northern hemisphere
    seasons_to_months = {
        'nh_winter': [12, 1, 2],
        'nh_spring': [3, 4, 5],
        'nh_summer': [6, 7, 8],
        'nh_fall': [9, 10, 11]
    }
    
    # Define month name mapping
    month_names = {
        1: 'J', 2: 'F', 3: 'M', 4: 'A', 5: 'M', 6: 'J',
        7: 'J', 8: 'A', 9: 'S', 10: 'O', 11: 'N', 12: 'D'
    }

    # Define number of days per month (assuming 28 days for February)
    days_per_month = {
        1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30,
        7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31
    }

    months = []

    # If no specific period is selected, all data will be used.
    if period is None:
        period_name = 'whole_year'
        months = list(range(1, 13))  # All months
    elif isinstance(period, int):
        period_name = month_names[period]
        months = [period]
    elif isinstance(period, str):
        # Check if the input is a single season or multiple seasons
        if 'and' in period:
            seasons = period.lower().split('and')
            period_name = ''
            for season in seasons:
                season = season.strip()
                months.extend(seasons_to_months.get(season, []))
                period_name += ''.join(month_names[m] for m in seasons_to_months.get(season, []))
        else:
            months = seasons_to_months.get(period.lower(), [])
            period_name = ''.join(month_names[m] for m in months)
    elif isinstance(period, list):
        period_name = ''.join(month_names[m] for m in period if m in month_names)
        months = period
    else:
        raise ValueError("Period must be None, an integer, a string representing a single season, "
                         "a string with multiple seasons separated by 'and', or a list of integers.")

    for k, ds in ds_dict_copy.items():
        if start_year and end_year:
            start_date = f'{start_year}-01-01'
            end_date = f'{end_year}-12-31'
            ds = ds.sel(time=slice(start_date, end_date))

        # If months are specified, select those months
        if months:
            month_mask = ds['time.month'].isin(months)
            ds = ds.where(month_mask, drop=True)

        # Store the original attributes of each variable
        original_attrs = {var: ds[var].attrs for var in ds.data_vars}

        # If yearly_sum is True, sum over 'time' dimension to get yearly sum
        if yearly_sum: # does only make sense for accumulative variables e.g. pr or tran
            attrs = ds.attrs
            # Multiply each value by the number of days in the respective month
            days = ds['time'].dt.days_in_month
            ds = (ds * days).resample(time='AS').sum(dim='time')
            sum_type = 'yearly_sum'
            ds.attrs = attrs
        else:
            sum_type = 'monthly_mean'

        # Reassign the original attributes back to each variable
        for var in ds.data_vars:
            ds[var].attrs = original_attrs[var]

        ds_dict_copy[k] = ds
        ds_dict_copy[k].attrs['months'] = period_name
        ds_dict_copy[k].attrs['yearly_sum'] = sum_type

    return ds_dict_copy


In [ ]:
# ======== Standardize ========
def standardize(ds_dict):
    '''
    Helper function to standardize datasets of a dictionary
    '''
    ds_dict_stand = {}
    
    for name, ds in ds_dict.items():
        attrs = ds.attrs
        ds_stand = (ds - ds.mean()) / ds.std()

        # Preserve variable attributes from the original dataset
        for var in ds.variables:
            if var in ds_stand.variables:
                ds_stand[var].attrs = ds[var].attrs

        ds_stand.attrs = attrs
        ds_dict_stand[name] = ds_stand
        
    return ds_dict_stand

In [ ]:
def check_args_and_get_info(ds_dict, variable):
    # Check the validity of input arguments
    if not isinstance(ds_dict, dict):
        raise TypeError("ds_dict must be a dictionary of xarray datasets.")
    if not all(isinstance(ds, xr.Dataset) for ds in ds_dict.values()):
        raise TypeError("All values in ds_dict must be xarray datasets.")
    if not isinstance(variable, str):
        raise TypeError('variable must be a string.')
        
    # Dictionary to store plot titles for each statistic
    titles = {"mean": "Mean", "std": "Standard deviation of yearly means", "min": "Minimum", "max": "Maximum", "median": "Median", "time": "Time", "space": "Space"}
    freq = {"mon": "Monthly"}
    
    long_name = {
        'Precipitation': 'Precipitation',
        'Total Runoff': 'Total Runoff',
        'Vapor Pressure Deficit': 'Vapor Pressure Deficit',
        'Evaporation Including Sublimation and Transpiration': 'Evapotranspiration',
        'Transpiration': 'Transpiration',
        'Leaf Area Index': 'Leaf Area Index',
        'Carbon Mass Flux out of Atmosphere Due to Gross Primary Production on Land [kgC m-2 s-1]': 'Gross Primary Production',
        'Total Liquid Soil Moisture Content of 1 m Column': '1 m Soil Moisture',
        'Total Liquid Soil Moisture Content of 2 m Column': '2 m Soil Moisture',
        'Runoff - Precipitation': 'Runoff - Precipitation',
        'Transpiration - Precipitation': 'Transpiration - Precipitation',
        '(Runoff + Transpiration) - Precipitation':  '(Runoff + Transpiration) - Precipitation',
        'ET - Precipitation':  'ET - Precipitation', 
        'Negative Runoff': 'Negative Runoff',
    }
   
    # Data information
    var_long_name = ds_dict[list(ds_dict.keys())[0]][variable].long_name
    period = f"{ds_dict[list(ds_dict.keys())[0]].attrs['period'][0]}-{ds_dict[list(ds_dict.keys())[0]].attrs['period'][1]}"
    experiment_id =  ds_dict[list(ds_dict.keys())[0]].experiment_id
    unit = ds_dict[list(ds_dict.keys())[0]][variable].units
    statistic_dim = ds_dict[list(ds_dict.keys())[0]].statistic_dimension
    statistic = ds_dict[list(ds_dict.keys())[0]].attrs['statistic']
    frequency = freq[ds_dict[list(ds_dict.keys())[0]].frequency]

    return var_long_name, period, unit, statistic_dim, statistic, experiment_id, titles, frequency

In [ ]:
def compute_ensemble(ds_dict_change):
    for key in ['Ensemble mean', 'Ensemble median']:
        if key in ds_dict_change:
            ds_dict_change.pop(key)

    # Drop 'member_id' coordinate if it exists in any of the datasets
    for ds_key in ds_dict_change:
        if 'member_id' in ds_dict_change[ds_key].coords:
            ds_dict_change[ds_key] = ds_dict_change[ds_key].drop('member_id')

    combined = xr.concat(ds_dict_change.values(), dim='ensemble')
    ds_dict_change['Ensemble median'] = getattr(combined, 'median')(dim='ensemble')
    
    return ds_dict_change

In [ ]:
import regionmask

def apply_region_mask(ds_dict):
    """
    Applies the AR6 land region mask to datasets in the provided dictionary and adds a region dimension.

    Args:
        ds_dict (dict): A dictionary of xarray datasets.

    Returns:
        dict: A new dictionary where keys are the same as in the input dictionary,
              and each value is an xarray Dataset with a region dimension added to each variable.
    """
    
    land_regions = regionmask.defined_regions.ar6.land
    ds_masked_dict = {}
    
    for ds_name, ds in ds_dict.items():
        ds_out = xr.Dataset()  # Initiate an empty Dataset for the masked data
        
        # Get attributes
        attrs = ds.attrs
        
        for var in ds:
            # Get the binary mask
            mask = land_regions.mask_3D(ds[var])
            
            var_attrs = ds[var].attrs

            # Multiply the original data with the mask to get the masked data
            masked_var = ds[var] * mask

            # Replace 0s with NaNs, if desired
            masked_var = masked_var.where(masked_var != 0)

            # Add the masked variable to the output Dataset
            ds_out[var] = masked_var
            
            ds_out[var].attrs = var_attrs
            
        # Add the attributes
        ds_out.attrs = attrs

        # Add the Dataset to the output dictionary
        ds_masked_dict[ds_name] = ds_out

    return ds_masked_dict

In [ ]:
def is_numeric(data):
    try:
        _ = data.astype(float)
        return True
    except (ValueError, TypeError):
        return False

def compute_change(ds_dict_hist, ds_dict_fut, var_rel_change=None):
    ds_dict_change = {}

    for name, ds_hist in ds_dict_hist.items():
        if name in ds_dict_fut:
            ds_future = ds_dict_fut[name]
            common_vars = set(ds_hist.data_vars).intersection(ds_future.data_vars)

            ds_change = ds_hist.copy(deep=True)
            
            if var_rel_change == 'all':
                var_rel_change = common_vars
                
            for var in common_vars:
                if is_numeric(ds_hist[var].data) and is_numeric(ds_future[var].data):
                    if var_rel_change is not None and var in var_rel_change:
                        # Compute relative change where ds_hist is not zero
                        rel_change = (ds_future[var] - ds_hist[var]) / ds_hist[var].where(ds_hist[var] != 0) * 100
                        ds_change[var].data = rel_change.data
                        ds_change[var].attrs['units'] = '%'
                    else:
                        # Compute absolute change
                        abs_change = ds_future[var] - ds_hist[var]
                        ds_change[var].data = abs_change.data

            ds_change.attrs = ds_future.attrs
            ds_dict_change[name] = ds_change

    return ds_dict_change

#### Compute statistics

In [ ]:
def calculate_spatial_mean(ds_dict):
    ds_dict_mean = {}
    for key, ds in ds_dict.items():
        attrs = ds.attrs
        for var in list(ds.data_vars.keys()):
            var_attrs = ds[var].attrs
            
            ds_dict_mean[key][var] = ds.mean(['lon', 'lat'])
            ds_dict_mean[key][var].attrs = var_attrs
        
        ds_dict_mean[key].attrs = attrs
        
    return ds_dict_mean

In [ ]:
def compute_statistic_single(ds, statistic, dimension, yearly_mean=True):
    if dimension == "time":
        stat_ds = getattr(ds, statistic)("time", keep_attrs=True, skipna=True)
        stat_ds.attrs['period'] = [str(ds.time.dt.year[0].values), str(ds.time.dt.year[-1].values)]
        
    if dimension == "space":
        # Assign the period attribute before grouping by year
        ds.attrs['period'] = [str(ds.time.dt.year[0].values), str(ds.time.dt.year[-1].values)]
        
        if yearly_mean:
            ds = ds.groupby('time.year').mean('time', keep_attrs=True, skipna=True)
            ds.attrs['mean'] = 'yearly mean'
            
        
        #get the weights, apply on data, and compute statistic
        weights = np.cos(np.deg2rad(ds.lat))
        weights.name = "weights"
        ds_weighted = ds.weighted(weights)
        stat_ds = getattr(ds_weighted, statistic)(("lon", "lat"), keep_attrs=True, skipna=True)
    
    stat_ds.attrs['statistic'] = statistic
    stat_ds.attrs['statistic_dimension'] = dimension

    return stat_ds

In [ ]:
def compute_statistic(ds_dict, statistic, dimension, start_year=None, end_year=None, yearly_mean=True):
    """
    Computes the specified statistic for each dataset in the dictionary.

    Args:
        ds_dict (dict): A dictionary of xarray datasets, where each key is the name of the dataset
            and each value is the dataset itself.
        statistic (str): The statistic to compute, which can be one of 'mean', 'std', 'min', 'var', or 'median'.
        dimension (str): The dimension to compute over, which can be 'time' or 'space'.
        start_year (str, optional): The start year of the period to compute the statistic over.
        end_year (str, optional): The end year of the period to compute the statistic over.

    Returns:
        dict: A dictionary with computed statistic for each dataset.
    """
    # Check the validity of input arguments
    if not isinstance(ds_dict, dict):
        raise TypeError("ds_dict must be a dictionary of xarray datasets.")
    if not all(isinstance(ds, xr.Dataset) for ds in ds_dict.values()):
        raise TypeError("All values in ds_dict must be xarray datasets.")
    if statistic not in ["mean", "std", "min", "max", "var", "median"]:
        raise ValueError(f"Invalid statistic '{statistic}' specified.")
    if dimension not in ["time", "space"]:
        raise ValueError(f"Invalid dimension '{dimension}' specified.")

    # Select period
    if start_year is not None and end_year is not None:
        select_period(ds_dict, start_year=start_year, end_year=end_year)
        
        
    # Use multiprocessing to compute the statistic for each dataset in parallel
    with mp.Pool() as pool:
        results = pool.starmap(compute_statistic_single, [(ds, statistic, dimension, yearly_mean) for ds in ds_dict.values()])

    return dict(zip(ds_dict.keys(), results))

In [ ]:
from itertools import permutations

def precompute_metrics(ds_dict, variables, metrics=['pearson']):
    # Initialize the results dictionary
    results_dict = {metric: {} for metric in metrics}
    
    for name, ds in ds_dict.items():
        # Create a DataFrame with all the variables
        df = pd.DataFrame({var: ds[var].values.flatten() for var in variables})
        
        # Define all pairs of variables
        pairs = list(permutations(variables, 2))  # <-- Change here
        args = [(df, var1, var2, metrics) for var1, var2 in pairs]

        # Use a multiprocessing pool to compute the metrics for all pairs
        with Pool() as p:
            results = p.map(compute_metrics_for_pair, args)
        
        # Store the results in the results_dict
        for var1, var2, metric_dict in results:
            for metric, value in metric_dict.items():
                # Ensure the keys exist in the dictionary
                results_dict[metric].setdefault(name, {}).setdefault(f'{var1}_{var2}', value)
    return results_dict

In [ ]:
def compute_stats(ds_dict):
    """
    Compute yearly mean of each variable in the dataset.

    Parameters:
    ds_dict (dict): The input dictionary of xarray.Dataset.

    Returns:
    dict: A dictionary where the keys are the dataset names and the values are another dictionary.
          This inner dictionary has keys as variable names and values as DataArray of yearly means.
    """
    stats = {}
    for model, ds in ds_dict.items():
        # Compute the yearly mean
        yearly_ds = ds.resample(time='1Y').mean()

        stats[model] = {}
        for var in yearly_ds.data_vars:
            # Compute the spatial mean
            spatial_mean = yearly_ds[var].mean(dim=['lat', 'lon'])
            
            # Store the yearly mean values
            stats[model][var] = spatial_mean
    return stats

In [ ]:
def compute_yearly_means(ds_dict):
    yearly_means_dict = {}

    # For each dataset, compute the yearly mean over the 'time', 'lat', and 'lon' dimensions
    for name, ds in ds_dict.items():  
        ds_yearly = ds.groupby('time.year').mean('time')    
        
        yearly_means_dict[name] = ds_yearly

    return yearly_means_dict

In [ ]:
def compute_yearly_regional_means(ds_dict_region):
    yearly_means_dict = {}

    # For each dataset, compute the yearly mean over the 'time', 'lat', and 'lon' dimensions
    for region, ds_dict in ds_dict_region.items():
        yearly_means_dict[region] = {}
        for ds_name, ds in ds_dict.items():
            # Compute the yearly mean
            ds_yearly = ds.groupby('time.year').mean('time')
            
            # Create weights
            weights = np.cos(np.deg2rad(ds.lat))
            # Apply the weights and calculate the spatial mean
            ds_weighted = ds_yearly.weighted(weights)
            yearly_means_dict[region][ds_name] = ds_weighted.mean(('lat', 'lon'))

    return yearly_means_dict

In [ ]:
def calculate_spatial_mean(ds_dict):
    ds_dict_mean = {}
    
    for key, ds in ds_dict.items():
        attrs = ds.attrs
        
        # Initialize a new Dataset for this key
        ds_dict_mean[key] = xr.Dataset()
        
        for var in list(ds.data_vars.keys()):
            var_attrs = ds[var].attrs
            
            ds_dict_mean[key][var] = ds[var].mean(['lon', 'lat'])
            ds_dict_mean[key][var].attrs = var_attrs
        
        ds_dict_mean[key].attrs = attrs
        
    return ds_dict_mean

In [ ]:
def calculate_spatial_mean_std(ds_dict):
    ds_stats = {}
    
    for key, ds in ds_dict.items():
        attrs = ds.attrs
        
        # Initialize a new Dataset for spatial statistics
        ds_stats[key] = xr.Dataset()
        
        for var in list(ds.data_vars.keys()):
            var_attrs = ds[var].attrs
            
            # Calculate the mean and standard deviation, skipping NaN values
            ds_stats[key][f'{var}'] = ds[var].mean(['lon', 'lat'], skipna=True)
            ds_stats[key][f'{var}'].attrs = var_attrs
            
            # Use a minimum count of 2 for standard deviation to ensure degrees of freedom > 0
            ds_stats[key][f'{var}_std'] = ds[var].std(['lon', 'lat'], skipna=True)
            ds_stats[key][f'{var}_std'].attrs = var_attrs
        
        ds_stats[key].attrs = attrs
        
    return ds_stats

In [ ]:
def compute_bgws(ds_dict):

    for model, ds in ds_dict.items():
        bgws = (ds['mrro']-ds['tran'])/ds['pr']

        # Replace infinite values with NaN
        bgws = xr.where(np.isinf(bgws), float('nan'), bgws)

        # Set all values above 2 and below -2 to NaN
        bgws = xr.where(bgws > 2, float('nan'), bgws)
        bgws = xr.where(bgws < -2, float('nan'), bgws)

        ds_dict[model]['bgws'] = bgws
        ds_dict[model]['bgws'].attrs = {'long_name': 'Blue Green Water Share',
                             'units': ''}
        
    return ds_dict

### Load netCDF files

In [ ]:
# ========= Define period, models and path ==============
#variables=['tas', 'pr', 'vpd', 'evspsbl', 'mrro', 'lmrso_1m', 'lmrso_2m', 'tran', 'lai', 'gpp', 'EI', 'wue']
variables=['tas', 'pr', 'vpd', 'evspsbl', 'mrro', 'mrso', 'tran', 'lai', 'gpp']
#variables=['pr', 'evspsbl', 'mrro', 'tran']

folder='preprocessed'

# ========= Use Dask to parallelize computations ==========
dask.config.set(scheduler='processes')

# Create dictionary using a dictionary comprehension and Dask
experiment_id = 'historical'
source_id = ['BCC-CSM2-MR', 'CAMS-CSM1-0', 'CanESM5-CanOE', 'CanESM5', 'CESM2-WACCM','CNRM-CM6-1', 'CNRM-ESM2-1','GFDL-ESM4','GISS-E2-1-G','MIROC-ES2L', 'MPI-ESM1-2-LR','NorESM2-MM','TaiESM1','UKESM1-0-LL']
ds_dict_hist = dask.compute({model: open_and_merge_datasets(folder, model, experiment_id, variables) for model in source_id})[0]

experiment_id = 'ssp126'
source_id = ['BCC-CSM2-MR', 'CAMS-CSM1-0', 'CanESM5-CanOE', 'CanESM5', 'CNRM-CM6-1', 'CNRM-ESM2-1','GFDL-ESM4','GISS-E2-1-G','MIROC-ES2L', 'MPI-ESM1-2-LR','UKESM1-0-LL']
ds_dict_ssp126 = dask.compute({model: open_and_merge_datasets(folder, model, experiment_id, variables) for model in source_id})[0]


experiment_id = 'ssp370'
source_id = ['BCC-CSM2-MR', 'CAMS-CSM1-0', 'CanESM5-CanOE', 'CanESM5', 'CESM2-WACCM','CNRM-CM6-1', 'CNRM-ESM2-1','GFDL-ESM4','GISS-E2-1-G','MIROC-ES2L', 'MPI-ESM1-2-LR','NorESM2-MM','TaiESM1','UKESM1-0-LL']
ds_dict_ssp370 = dask.compute({model: open_and_merge_datasets(folder, model, experiment_id, variables) for model in source_id})[0]


experiment_id = 'ssp585'
source_id = ['BCC-CSM2-MR', 'CAMS-CSM1-0', 'CanESM5-CanOE', 'CanESM5', 'CNRM-CM6-1', 'CNRM-ESM2-1','GFDL-ESM4','GISS-E2-1-G','MIROC-ES2L', 'MPI-ESM1-2-LR','UKESM1-0-LL']
ds_dict_ssp585 = dask.compute({model: open_and_merge_datasets(folder, model, experiment_id, variables) for model in source_id})[0]

In [ ]:
# ============= Have a look into the data ==============
#print(ds_dict_ssp126.keys())
#ds_dict_hist[list(ds_dict_region_change.keys())[8]]

### Select period

In [ ]:
#'nh_winter': [12, 1, 2],'nh_spring': [3, 4, 5],'nh_summer': [6, 7, 8], 'nh_fall': [9, 10, 11]

In [ ]:
ds_dict_hist_period = select_period(ds_dict_hist, start_year=1985, end_year=2014, period=None, yearly_sum=False)

In [ ]:
ds_dict_ssp126_period = select_period(ds_dict_ssp126, start_year=2071, end_year=2100, period=None, yearly_sum=False)

In [ ]:
ds_dict_ssp370_period = select_period(ds_dict_ssp370, start_year=2071, end_year=2100, period=None, yearly_sum=False)

In [ ]:
ds_dict_ssp585_period = select_period(ds_dict_ssp585, start_year=2071, end_year=2100, period=None, yearly_sum=False)

### Compute statistics

In [ ]:
# ========= Compute statistic for plot ===============
ds_dict_hist_period_metric = compute_statistic(ds_dict_hist_period, 'mean', 'time')

In [ ]:
ds_dict_ssp126_period_metric = compute_statistic(ds_dict_ssp126_period, 'mean', 'time')

In [ ]:
# ========= Compute statistic for plot ===============
ds_dict_ssp370_period_metric = compute_statistic(ds_dict_ssp370_period, 'mean', 'time')

In [ ]:
ds_dict_ssp585_period_metric = compute_statistic(ds_dict_ssp585_period, 'mean', 'time')

### Compute BGWS

In [ ]:
ds_dict_hist_period_metric = compute_bgws(ds_dict_hist_period_metric)

In [ ]:
ds_dict_hist_period_ = compute_bgws(ds_dict_hist_period)

In [ ]:
ds_dict_ssp126_period_metric = compute_bgws(ds_dict_ssp126_period_metric)

In [ ]:
ds_dict_ssp370_period_metric = compute_bgws(ds_dict_ssp370_period_metric)

In [ ]:
ds_dict_ssp585_period_metric = compute_bgws(ds_dict_ssp585_period_metric)

### Select regions

In [ ]:
ds_dict_hist_period_metric_regions = {}
ds_dict_hist_period_metric_regions = apply_region_mask(ds_dict_hist_period_metric)

In [ ]:
ds_dict_hist_period_ensemble_regions = {}
ds_dict_hist_period_ensemble_regions = apply_region_mask(ds_dict_hist_period_ensemble)

In [ ]:
ds_dict_ssp126_period_metric_regions = {}
ds_dict_ssp126_period_metric_regions = apply_region_mask(ds_dict_ssp126_period_metric)

In [ ]:
ds_dict_ssp370_period_metric_regions = {}
ds_dict_ssp370_period_metric_regions = apply_region_mask(ds_dict_ssp370_period_metric)

In [ ]:
ds_dict_ssp585_period_metric_regions = {}
ds_dict_ssp585_period_metric_regions = apply_region_mask(ds_dict_ssp585_period_metric)

### Compute regional mean

In [ ]:
# Compute spatial mean of regional data
ds_dict_hist_period_metric_regional_mean = {}
ds_dict_hist_period_metric_regional_mean = calculate_spatial_mean(ds_dict_hist_period_metric_regions)

In [ ]:
ds_dict_hist_period_ensemble_regional_mean = {}
ds_dict_hist_period_ensemble_regional_mean = calculate_spatial_mean(ds_dict_hist_period_ensemble_regions)

In [ ]:
ds_dict_hist_period_ensemble_regional_mean = ds_dict_hist_period_ensemble_regional_mean['Ensemble mean']

In [ ]:
ds_dict_hist_period_metric_regional_mean_std = {}
ds_dict_hist_period_metric_regional_mean_std = calculate_spatial_mean_std(ds_dict_hist_period_metric_regions)

In [ ]:
# Compute spatial mean of regional data
ds_dict_ssp126_period_metric_regional_mean = {}
ds_dict_ssp126_period_metric_regional_mean = calculate_spatial_mean(ds_dict_ssp126_period_metric_regions)

In [ ]:
# Compute spatial mean of regional data
ds_dict_ssp370_period_metric_regional_mean = {}
ds_dict_ssp370_period_metric_regional_mean = calculate_spatial_mean(ds_dict_ssp370_period_metric_regions)

In [ ]:
# Compute spatial mean of regional data
ds_dict_ssp585_period_metric_regional_mean = {}
ds_dict_ssp585_period_metric_regional_mean = calculate_spatial_mean(ds_dict_ssp585_period_metric_regions)

### Test BGWS robustness regionally

In [ ]:
import scipy.stats as stats

def calculate_ensemble_metrics(ds_dict_regional_mean, variable='bgws'):
    """
    Calculate the ensemble mean, its uncertainty (95% confidence interval), SEM, other spread metrics, 
    and the percentage of models agreeing on the sign of the mean for the 'bgws' variable in each region.
    :param ds_dict_regional_mean: Dictionary of xarray Datasets with regional mean values, keys are model names.
    :param variable: The variable to analyze, default is 'bgws'.
    :return: DataFrame with regions, ensemble mean, confidence interval, SEM, spread metrics, and sign agreement.
    """
    ensemble_data = []

    # Combining datasets into a single dataset
    combined_ds = xr.concat([ds[variable] for ds in ds_dict_regional_mean.values()], dim='model')

    # Calculate ensemble mean and spread metrics
    ensemble_mean = combined_ds.mean(dim='model')
    std_dev = combined_ds.std(dim='model')
    iqr = combined_ds.quantile(0.75, dim='model') - combined_ds.quantile(0.25, dim='model')

    # Calculate confidence interval (95%)
    sem = std_dev / np.sqrt(len(ds_dict_regional_mean))  # Standard Error of the Mean
    confidence_interval = stats.t.interval(0.95, len(ds_dict_regional_mean) - 1, loc=ensemble_mean, scale=sem)

    # Convert to DataFrame
    for region_number, region_name, mean, ci, std, iqr_val in zip(ensemble_mean.region.values, ensemble_mean.names.values, ensemble_mean.values, np.transpose(confidence_interval), std_dev.values, iqr.values):
        ensemble_data.append({
            'Region Number': region_number, 
            'Region': region_name, 
            'Ensemble Mean': mean, 
            'Lower CI': ci[0], 
            'Upper CI': ci[1],
            'Standard Deviation': std,
            'IQR': iqr_val
        })

    ensemble_df = pd.DataFrame(ensemble_data)

    # Sort DataFrame
    sorted_df = ensemble_df.sort_values(by=['Standard Deviation'], ascending=[True])

    return sorted_df

In [ ]:
ensemble_metrics_df = calculate_ensemble_metrics(ds_dict_hist_period_metric_regional_mean, 'bgws')

In [ ]:
ensemble_metrics_df

### Split positive and negative BGWS regions

In [ ]:
def compute_ensemble_metric(ds_dict, average='mean'):
    """
    Compute ensemble mean or median for a dictionary of xarray Datasets and preserve attributes.
    
    :param ds_dict: Dictionary of xarray Datasets.
    :param average: Type of ensemble statistic to compute ('mean' or 'median').
    :return: xarray Dataset with ensemble statistic and preserved attributes.
    """
    # Remove existing ensemble mean or median if present
    for key in ['Ensemble mean', 'Ensemble median']:
        ds_dict.pop(key, None)  # Use pop with None as default to avoid KeyError

    # Concatenate datasets along a new 'ensemble' dimension
    combined = xr.concat(ds_dict.values(), dim='ensemble')

    # Compute ensemble statistic
    ensemble_metric_ds = getattr(combined, average)(dim='ensemble')

    # Copy attributes from the original dataset
    first_ds = next(iter(ds_dict.values()))
    ensemble_metric_ds.attrs = first_ds.attrs

    # Copy attributes for each variable
    for var in ensemble_metric_ds.variables:
        if var in first_ds.variables:
            ensemble_metric_ds[var].attrs = first_ds[var].attrs

    # Assign ensemble statistic as a global attribute
    ensemble_metric_ds.attrs['ensemble_stat'] = average

    return ensemble_metric_ds

In [ ]:
ensemble_mean_ds = compute_ensemble_metric(ds_dict_hist_period_metric_regional_mean)

In [ ]:
def split_dataset_by_bgws_sign(dataset, variable='bgws'):
    """
    Split the dataset into two based on the sign of the specified variable's values.

    :param dataset: xarray Dataset.
    :param variable: The variable to check the sign (default is 'bgws').
    :return: Two xarray Datasets - one with positive and the other with negative values of the variable.
    """
    
    # Split dataset based on the sign of the 'bgws' variable
    positive_ds = dataset.where(dataset[variable] > 0, drop=True)
    negative_ds = dataset.where(dataset[variable] < 0, drop=True)

    return positive_ds, negative_ds

In [ ]:
positive_bgws_ds, negative_bgws_ds = split_dataset_by_bgws_sign(ensemble_mean_ds)

### Cluster regions based on BGWS

In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler

In [ ]:
def cluster_regions_gmm(ds, max_clusters=10, important_vars=None, importance_factor=1):
    """
    Clusters regions using a Gaussian Mixture Model based on their numeric variables in a single dataset.
    Important variables can be given increased weight by duplicating them.
    Data is standardized before clustering. Returns a DataFrame with region names and their corresponding cluster numbers.

    :param ds: An xarray.Dataset object.
    :param max_clusters: The maximum number of clusters to test for the elbow method.
    :param important_vars: List of variables to be given increased importance.
    :param importance_factor: Factor to determine how many times to duplicate important variables.
    :return: A pandas DataFrame with columns 'Region' and 'Cluster'.
    """
    # Convert the xarray Dataset to a pandas DataFrame
    df = ds.to_dataframe().reset_index()
    df = df.set_index('region')

    # Select only numeric columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    df_numeric = df[numeric_cols]

    # Drop rows with NaN values
    df_numeric = df_numeric.dropna()

    # Duplicate important variables to increase their weight
    if important_vars and importance_factor > 1:
        for var in important_vars:
            if var in df_numeric.columns:
                for i in range(importance_factor - 1):  # We already have one set of important vars
                    df_numeric[f'{var}_dup_{i}'] = df_numeric[var]

    # Find the optimal number of clusters using Gaussian Mixture Model
    bic_scores = []
    K = range(1, max_clusters + 1)
    for k in K:
        gmm = GaussianMixture(n_components=k, n_init=10, random_state=42)
        gmm = gmm.fit(df_numeric)
        bic_scores.append(gmm.bic(df_numeric))

    # Plot the BIC scores
    plt.plot(K, bic_scores, 'bx-')
    plt.xlabel('k (number of components)')
    plt.ylabel('BIC score')
    plt.title('BIC Scoring for Gaussian Mixture Model')
    plt.show()

    # Ask user for the optimal number of clusters
    n_clusters = int(input("Enter the optimal number of clusters: "))

    # Use the Gaussian Mixture Model to find clusters with the optimal number of clusters
    gmm = GaussianMixture(n_components=n_clusters, n_init=10, random_state=42)
    gmm = gmm.fit(df_numeric)

    # Predict the cluster for each region
    cluster_labels = gmm.predict(df_numeric)

    # Create a DataFrame with region identifiers and their assigned clusters
    clustered_df = pd.DataFrame({
        'Region': df.index,  # Adjust this if your region identifier column is named differently
        'Cluster': cluster_labels
    }).sort_values('Cluster')

    # Merge the original DataFrame (with variable values) with the cluster assignments
    merged_df = clustered_df.merge(df[numeric_cols + ['abbrevs', 'names']], left_on='Region', right_index=True)

    return merged_df

In [ ]:
important_variables = ['bgws']
clustered_negative_bgws_ds = cluster_regions_gmm(negative_bgws_ds, important_vars=None, importance_factor=2)

In [ ]:
# Set pandas options to display all columns
pd.set_option('display.max_columns', None)

# Set pandas options to display all rows
pd.set_option('display.max_rows', None)

# If the table is still too wide, you can also adjust the max column width
pd.set_option('display.max_colwidth', None)

# If your table has many columns and you want to see them all at once without truncation, you can set a high threshold for column width
pd.set_option('display.width', 1000)

In [ ]:
clustered_negative_bgws_ds

In [ ]:
clustered_positive_bgws_ds = cluster_regions_gmm(positive_bgws_ds)

In [ ]:
clustered_positive_bgws_ds

In [ ]:
clustered_positive_bgws_ds

### Plot regional var change

In [ ]:
def calculate_ticks(min_val, max_val, base=20):
    """Calculate tick positions for the y-axis."""
    # Calculate the 10% of the min and max values
    offset_min = abs(min_val) * 0.1
    offset_max = max_val * 0.1

    # Find the next "round" number beyond min and max values based on the base
    lower_tick = base * np.floor(min_val / base)
    upper_tick = base * np.ceil(max_val / base)

    # Ensure lower_tick is not more than 10% lower than min_val
    if min_val - lower_tick > offset_min:
        lower_tick += base

    # Ensure upper_tick is not more than 10% higher than max_val
    if upper_tick - max_val > offset_max:
        upper_tick -= base

    # If the lower_tick or upper_tick is equal to the base, divide the base by 2
    if lower_tick >= -base or upper_tick <= base:
        base = base / 2
        # Recalculate the ticks with the new base
        lower_tick = base * np.floor(min_val / base)
        upper_tick = base * np.ceil(max_val / base)

    # Include zero and extend to the next round number beyond the data's min and max
    ticks = [lower_tick] if lower_tick < 0 else []
    ticks += [0]  # Always include zero
    ticks += [upper_tick] if upper_tick > 0 else []

    # Generate intermediate ticks between the round numbers
    intermediate_ticks = np.arange(lower_tick + base, upper_tick, base)
    ticks.extend(intermediate_ticks)

    return sorted(ticks)

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import BoundaryNorm

# Define start and end colors for both gradients
deep_blue = (20/255, 110/255, 180/255)  
light_blue = (180/255, 215/255, 255/255)
deep_green = (14/255, 119/255, 14/255) 
light_green = (160/255, 220/255, 140/255)

# Create custom colormaps
blue_cmap = LinearSegmentedColormap.from_list("blue_cmap", [light_blue, deep_blue], N=4)
green_cmap = LinearSegmentedColormap.from_list("green_cmap", [deep_green, light_green], N=4)

# Sample colors from colormaps
blue_colors = [blue_cmap(i) for i in np.linspace(0, 1, 4)]
green_colors = [green_cmap(i) for i in np.linspace(0, 1, 4)]

# Combine both gradients
combined_grad = green_colors + blue_colors 

# Define boundaries
boundaries = [-0.1, -0.75, -0.05, -0.25, 0, 0.25, 0.05, 0.75, 0.1]
norm = BoundaryNorm(boundaries, len(combined_grad), clip=True)

cmap_name = 'BGWS colormap'
bgws_cm = LinearSegmentedColormap.from_list(cmap_name, combined_grad, N=len(combined_grad))

# To test and display the colormap
fig, ax = plt.subplots(figsize=(6, 1))
ax.set_title(cmap_name)
plt.imshow(np.linspace(0, 1, 256).reshape(1, -1), aspect='auto', cmap=bgws_cm)
plt.axis('off')
plt.show()

In [ ]:
def plot_region_change(ds_dict_change, selected_indices="ALL", save_fig=True):
    
    # Compute ensemble data
    ds_dict_change = compute_ensemble(ds_dict_change)

    # Determine the type of change: relative or absolute
    change = 'rel_change' if ds_dict_change[list(ds_dict_change.keys())[0]]['pr'].units == '%' else 'abs_change'

    # Extract models and variables information
    models = list(ds_dict_change.keys())
    ensemble_median = ds_dict_change['Ensemble median']
    variables = [var for var in ensemble_median.data_vars.keys() if var != 'bgws']
    experiment_id = ds_dict_change[list(ds_dict_change.keys())[0]].experiment_id
    description = ds_dict_change[list(ds_dict_change.keys())[0]].description
    # Check for 'period' and 'yearly_sum' attributes, set defaults if not found
    months = ds_dict_change[list(ds_dict_change.keys())[0]].months
    yearly_sum = ds_dict_change[list(ds_dict_change.keys())[0]].yearly_sum

            

    # Create a map for variable display names
    var_map = {
        'tas': 'T', 'vpd': 'VPD', 'gpp': 'GPP', 'pr': 'P', 'mrro': 'R',
        'evspsbl': 'ET', 'tran': 'Tran', 'lai': 'Lai', 'mrso': 'SM'
    }
    display_vars = [
        f"$\Delta\, \mathrm{{\it{{{var_map[var]}}}}}$" if var in var_map else var for var in variables
    ]

    # Handle selection of indices
    selected_indices = (ensemble_median.region.values.tolist()
                        if selected_indices == "ALL" else selected_indices)

    # Setup plot grid
    nregions = len(selected_indices)
    ncols = 9
    nrows = (nregions + ncols - 1) // ncols 
    
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(8 * ncols, 7 * nrows))

    # Setup plot grid
    #nrows = len(selected_indices) // 9 + (len(selected_indices) % 9 > 0)
    #fig, axes = plt.subplots(nrows=nrows, ncols=9, figsize=(8 * 9, 7 * nrows), squeeze=False)
    
    # bgws_colormap
    bgws_vals = np.array([ds_dict_change[model]['bgws'].values for model in models])
    bgws_min, bgws_max = np.nanmin(bgws_vals), np.nanmax(bgws_vals)
    norm = plt.Normalize(vmin=-0.1, vmax=0.1)
    
    # Preliminary settings
    threshold = 100  # Set the threshold for y-axis limits
    capped_value = 105  # The value to assign to capped data points

    if selected_indices == "ALL":
        selected_indices = ds_dict_change['Ensemble median'].region.values.tolist()
        
    # Iterate over each region to plot data
    for ridx, region_idx in enumerate(selected_indices):
        ax = axes.flatten()[ridx]
        max_change = 0  # Track the maximum change for y-axis limits   
        min_change = 0 
        
        # Collect all y-values to determine if any exceed the threshold
        all_y_vals = []
        
        for idx, model in enumerate(models):
            model_data = ds_dict_change[model]
            y = [
                model_data[var].sel(region=region_idx).values
                if var in ds_dict_change[model].data_vars else float('nan')
                for var in variables
            ]
            all_y_vals.extend(y)
            
            # Ensure y is a NumPy array for element-wise comparison
            y_array = np.array(y)
            
            # Cap the values at the threshold
            y_capped = np.clip(y_array, -threshold, threshold)
            # Set values that were above or below the threshold
            y_capped = np.where(y_array > threshold, capped_value, y_capped)  # For values above the threshold
            y_capped = np.where(y_array < -threshold, -capped_value, y_capped)  # For values below the threshold
            
            # Update max_change and min_change
            max_change = max(max_change, np.nanmax(y))  
            min_change = min(min_change, np.nanmin(y)) 
            
            # Set plotting of bgws
            x = range(len(display_vars))
            bgws_val = model_data['bgws'].sel(region=region_idx).values if 'bgws' in model_data.data_vars else None
            linestyle = '-' if bgws_val is not None and bgws_val >= 0 else '--'
            color = bgws_cm(norm(bgws_val))
            
            # Plot all models
            if model != "Ensemble median":
                ax.plot(x, y_capped, linestyle=linestyle, color=color, linewidth=1.0, alpha=0.6, zorder=1)
                for xi, yi in zip(x, y_capped):
                    if not np.isnan(yi):
                        ax.text(xi, yi, str(idx + 1), ha='center', va='center', fontsize=16, color=color, fontweight='bold', zorder=2)
            else: # Plot Ensemble median
                ax.scatter(x, y_capped, marker='D', edgecolors='red', s=150, label=model, zorder=3, facecolors='none', lw=2)
        
        
        # Set plot titles and labels
        ax.set_title(ensemble_median.names.sel(region=region_idx).values, fontsize=20)
        ax.set_xticks(range(len(display_vars)))
        ax.set_xticklabels(display_vars, rotation=0, fontsize=18)  
        y_label = 'End of century response [%]' if change == 'rel_change' else 'End of century response'
        ax.set_ylabel(y_label, fontsize=18)
        
        # Set y-axis limits dynamically based on data spread
        lower_limit = max(-threshold, min_change) 
        upper_limit = min(threshold, max_change) 

        # Add 10 on y-axis if change exceeds threshold
        extra_space = 10  # 10 percent extra space
        
        # Plot dashed line, extent and add arrows to y-axis if change > threshold
        # Max Change
        if max_change > threshold:
            ax.axhline(threshold, color='grey', linestyle='--', linewidth=0.5, zorder=1)
            ax.plot(0, threshold+10, "^k", transform=ax.get_yaxis_transform(), clip_on=False)

        # Min Change    
        if min_change < -threshold:
            ax.axhline(-threshold, color='grey', linestyle='--', linewidth=0.5, zorder=1)
            ax.plot(0, -threshold-10, "vk", transform=ax.get_yaxis_transform(), clip_on=False)
            
        # Set y-limits 
        #ax.set_ylim(lower_limit - extra_space, upper_limit + extra_space)
        ax.set_ylim(lower_limit * 1.1, upper_limit * 1.1)
        # Define the tick positions, ensuring they do not go beyond the max/min values or thresholds
        # Calculate and set y-axis ticks
        y_ticks = calculate_ticks(lower_limit, upper_limit)
        ax.set_yticks(y_ticks)
        
        ax.axhline(0, color='grey', linewidth=0.5)
        ax.tick_params(axis='x', length=0)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(True)
        ax.yaxis.set_tick_params(labelsize=14)
    
    
    # Caption and figure saving
    caption = f"{description} (2071-2100) - Historical (1985-2014)"
    fig.text(0.52, 1.04, caption, ha='center', va='top', fontsize=45, wrap=True, weight='bold')
    
    # Layout adjustments and legend
    plt.subplots_adjust(wspace=0.4, hspace=0.7)
    
    # Remove the empty plot in the last row and third column
    axes.flatten()[-1].remove()
    
     # Define positions for the legend and colorbar
    legend_position = [0.8, 0.04, 0.3, 0.15]  # [x0, y0, width, height]
    colorbar_position = [0.905, 0.055, 0.09, 0.015]

    # Add legend directly to the figure
    legend_ax = fig.add_axes(legend_position, frame_on=False)
    legend_elements = [plt.Line2D([0], [0], marker='D', markeredgecolor='red', markerfacecolor='none', label='Ensemble median', markersize=10, linestyle='None', lw=2)]
    for idx, model in enumerate(models):
        if model != "Ensemble median":
            legend_elements.append(plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='w', label=f"{idx + 1}: {model}", markersize=10))
    
    legend = legend_ax.legend(handles=legend_elements, fontsize=14, ncol=2, loc='center')
    legend_ax.axis('off')

    # Add the colorbar below the legend
    cbar_ax = fig.add_axes(colorbar_position)
    cbar = fig.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=bgws_cm), cax=cbar_ax, orientation='horizontal', extend='both')
    cbar.set_label("$\Delta$ Blue-Green Water Share", fontsize=16, weight='bold')

    # Define tick locations
    tick_locs = [-0.1, -0.075, -0.05, -0.025, 0, 0.025, 0.05, 0.075, 0.1]

    # Define tick labels
    tick_labels = ["-0.1", "-0.075", "-0.05", "-0.025", "0", "0.025", "0.05", "0.075", "0.1"]

    # Set the ticks and tick labels
    cbar.set_ticks(tick_locs)
    cbar.set_ticklabels(tick_labels)

    cbar.ax.tick_params(labelsize=14)

    # Adjust layout to prevent overlap
    plt.tight_layout()
    
    # Save figure
    if save_fig:
        savepath = os.path.join('../..', 'results', 'CMIP6', 'comparison', 'regional_var_change')
        os.makedirs(savepath, exist_ok=True)
        filename = f'regional_var_{change}_{experiment_id}_{months}_{yearly_sum}.pdf'
        filepath = os.path.join(savepath, filename)
        fig.savefig(filepath, dpi=600, bbox_inches='tight', format='pdf')
    else:
        filepath = 'Figure not saved. If you want to save the figure add save_fig=True to the function call'

    return filepath

In [ ]:
plot_region_change(ds_dict_region_change_ssp126, selected_indices="ALL", save_fig=True)

In [ ]:
plot_region_change(ds_dict_region_change_ssp370, selected_indices="ALL", save_fig=True)

In [ ]:
plot_region_change(ds_dict_region_change_ssp585, selected_indices="ALL", save_fig=True)

In [ ]:
# Clustern und bgws nicht den change plotten, da ich den plot sonst sehr schwer zu 
# verstehen finde. Grün kann nämich bedeuten, dass sehr positive (runoff dominiert) bgws regionen etwas
# negativer werden. Eventuell doch absolute veränderung plotten. Gleich größe relative Abnahme von runoff 
# und Transpiration können bedeuten, dass runoff viel mehr abnimmt in absoluten Zahlen und deswegen
# BGWS abnimmt (grün)

### Compute global median change

In [ ]:
# Compute Change
ds_dict_change = compute_change(ds_dict_hist_period_metric, ds_dict_ssp126_period_metric, var_rel_change=None)

In [ ]:
# Compute ensemble median for the change
ds_dict_change_ensemble = compute_ensemble(ds_dict_change)

In [ ]:
# Only use ensemble median for further analysis
ds_ensmed_glob = ds_dict_change_ensemble['Ensemble median']

In [ ]:
ds_ensmed_glob

In [ ]:
ds_dict_region_change_ssp126_ensemble = compute_ensemble(ds_dict_region_change_ssp126)

In [ ]:
ds = ds_dict_region_change_ssp126_ensemble['Ensemble median']

In [ ]:
ds

### Plot clusters on map¶

In [ ]:
import cartopy.crs as ccrs
from cartopy.feature import ShapelyFeature
import regionmask
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon

In [ ]:
from shapely.geometry import MultiPolygon, Polygon

def split_polygon(polygon, meridian=180):
    """
    Splits a Shapely polygon into two polygons at a specified meridian
    """
    minx, miny, maxx, maxy = polygon.bounds
    if maxx > meridian and minx < -meridian:
        # Polygon crosses the antimeridian
        left_poly = []
        right_poly = []
        for x, y in polygon.exterior.coords:
            if x >= meridian:
                right_poly.append((x - 360, y))  # Wraparound for the right side
            else:
                left_poly.append((x, y))
        return [Polygon(left_poly), Polygon(right_poly)]
    else:
        return [polygon]  # Wrap the single polygon in a list for consistency

In [ ]:
def plot_clusters_and_regions(ds_ensmed_glob, ds, clustered_df):
    # Initialize the plot with a cartopy projection
    fig = plt.figure(figsize=(30, 15))
    ax_main = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    
    # Plot the 'bgws' variable from the dataset
    ds_ensmed_glob['bgws'].plot(ax=ax_main, vmin=-0.2, vmax=0.2, cmap=bgws_cm, transform=ccrs.PlateCarree(), add_colorbar=False)
    
    # Add coastlines and gridlines
    ax_main.coastlines()
    ax_main.tick_params(axis='both', which='major', labelsize=20)
    gridlines = ax_main.gridlines(draw_labels=True, color='black', alpha=0.2, linestyle='--')
    gridlines.top_labels = gridlines.right_labels = False
    gridlines.xlabel_style = {'size': 18}
    gridlines.ylabel_style = {'size': 18}
    
    # Get region bounds using regionmask
    land_regions = regionmask.defined_regions.ar6.land
    
    # Create a mapping from region names to region numbers
    region_name_to_number = dict(zip(ds.names.values, ds.region.values))
    
    # Map the region names to the same order as ds.names.values
    region_to_cluster_map = dict(zip(clustered_df['Region'], clustered_df['Cluster']))

    # Now create a new column in ds that maps the region names to cluster numbers
    # This assumes ds.names.values has the same region names as in clustered_df['Region']
    ds['Cluster'] = [region_to_cluster_map[name] for name in ds.names.values]

    # Convert the 'Cluster' DataArray to a NumPy array and get unique values
    unique_clusters = np.unique(ds['Cluster'].values)

    # Prepare colors for clusters - this assumes a finite number of clusters
    cluster_colors = plt.cm.tab20b(np.linspace(0, 1, len(unique_clusters)))


    # Loop over the regions and plot the cluster numbers with abbreviations
    for region_name, cluster_number in zip(ds.names.values, ds['Cluster']):
        reg_num = region_name_to_number[region_name]
        region_polygons = land_regions[reg_num].polygon
        
        region_abbr = ds.abbrevs.values[ds.region.values == reg_num][0]  # Assuming this gives us the correct abbreviation
        cluster_color = cluster_colors[cluster_number]  # Get the color for the cluster
        
        # Fetch the polygon or polygons for this region
        region_obj = land_regions[reg_num]
        if hasattr(region_obj, 'polygons'):
            # If the attribute is 'polygons', we assume it's iterable (e.g., a list of Polygon objects)
            region_polygons = region_obj.polygons
        elif hasattr(region_obj, 'polygon'):
            # If there's only one Polygon, we wrap it in a list to make it iterable
            region_polygons = [region_obj.polygon]
        else:
            raise AttributeError(f"The region object does not have 'polygons' or 'polygon' attribute.")

        for region_polygon in region_polygons:
            # If the polygon crosses the antimeridian, split it
            split_polys = split_polygon(region_polygon)

            # Handle both Polygon and MultiPolygon types after splitting
            for poly in split_polys:
                if isinstance(poly, Polygon):
                    features_to_plot = [poly]
                elif isinstance(poly, MultiPolygon):
                    features_to_plot = list(poly.geoms)
                else:
                    raise TypeError(f"Unhandled geometry type: {type(poly)}")

                for feature_poly in features_to_plot:
                    feature = ShapelyFeature([feature_poly], ccrs.PlateCarree(), edgecolor=cluster_color, facecolor='none', linewidth=2)
                    ax_main.add_feature(feature)

            # Calculate the centroid for text placement using features_to_plot
            centroids = [feature_poly.centroid for feature_poly in features_to_plot]
    
            text_lon, text_lat = max(centroids, key=lambda c: c.x).coords[0]  # Use the easternmost centroid
        
            # Ensure cluster_number is a plain integer if it's a single-item array or DataArray
            if isinstance(cluster_number, np.ndarray) and cluster_number.size == 1:
                cluster_number = cluster_number.item()  # Converts a one-element array to a scalar
            elif isinstance(cluster_number, xr.DataArray) and cluster_number.ndim == 0:
                cluster_number = cluster_number.values.item()  # Gets the scalar value from a 0-dim DataArray

            # Annotate the cluster number for each region
            ax_main.text(text_lon, text_lat, f"{region_abbr}\n{cluster_number}",
                         horizontalalignment='center', verticalalignment='center', transform=ccrs.PlateCarree(),
                         fontsize=20, bbox=dict(facecolor='white', alpha=0.5, edgecolor='none'))

    plt.show()

In [ ]:
plot_clusters_and_regions(ds_ensmed_glob, ds, clustered_df)

In [ ]:
def calculate_ensemble_average_per_cluster(clustered_df, ds_dict):
    """
    Calculate the ensemble average for each variable across all models, grouped by cluster,
    and append the region names corresponding to each cluster.

    :param clustered_df: DataFrame with regions and their corresponding cluster assignments.
    :param ds_dict: Dictionary of xarray.Dataset objects with percentage changes for each model.
    :return: A DataFrame with the ensemble mean for each variable for each cluster, along with region names.
    """
    # Aggregate data across all models
    ensemble_data = []

    # Extract all variables from the first dataset as a reference
    reference_ds = list(ds_dict.values())[0]
    variables = list(reference_ds.data_vars)

    for var in variables:
        # Check if the variable exists in all models
        if all(var in ds.data_vars for ds in ds_dict.values()):
            var_all_models = [ds[var].to_dataframe().reset_index()[['region', var]] for ds in ds_dict.values()]
            var_combined = pd.concat(var_all_models, axis=0)
            ensemble_mean = var_combined.groupby('region').mean()
            ensemble_data.append(ensemble_mean)

    # Combine the ensemble means for all variables
    combined_ensemble = pd.concat(ensemble_data, axis=1)

    # Merge with cluster assignments using the region index
    combined_ensemble.reset_index(inplace=True)
    combined_ensemble = combined_ensemble.merge(clustered_df, left_on='region', right_index=True)

    # Group by cluster and calculate the mean for each variable, excluding 'region'
    final_stats_df = combined_ensemble.drop(columns=['region']).groupby('Cluster').mean(numeric_only=True)

    # Append region names to each cluster
    region_names = reference_ds.abbrevs.values
    clustered_regions = clustered_df.reset_index().groupby('Cluster')['index'].apply(
        lambda x: [region_names[i] for i in x])
    final_stats_df['Regions'] = clustered_regions
    
    return final_stats_df

In [ ]:
average_changes_per_cluster = calculate_ensemble_average_per_cluster(clustered_df, ds_dict_region_change_ssp126)

In [ ]:
average_changes_per_cluster

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

def plot_cluster_distributions(clustered_df, ensemble_ds):
    # Convert the xarray Dataset to a pandas DataFrame
    ensemble_df = ensemble_ds.to_dataframe().reset_index()

    # Multiply 'bgws' by 100
    ensemble_df['bgws'] *= 100

    # Melt the DataFrame to long-form for plotting
    long_df = ensemble_df.melt(id_vars=['region', 'abbrevs', 'names'],
                               var_name='Variable', value_name='Value')

    # Merge with cluster assignments
    long_df = long_df.merge(clustered_df, left_on='names', right_on='Region')

    # Number of unique clusters
    num_clusters = clustered_df['Cluster'].nunique()

    # Calculate the layout for subplots (3 columns)
    num_cols = 3
    num_rows = np.ceil(num_clusters / num_cols).astype(int)

    # Create a figure with subplots
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 5 * num_rows), squeeze=False)
    axes = axes.flatten()

    # Define markers for regions
    markers = ['o', 's', 'D', '^', 'v', '<', '>', 'p', '*', 'h', 'H', 'X']  # All filled markers
    unique_abbrevs = long_df['abbrevs'].unique()
    marker_dict = {abbrev: markers[i % len(markers)] for i, abbrev in enumerate(unique_abbrevs)}

    # Iterate over each cluster to create a scatter plot
    for cluster_num in range(num_clusters):
        ax = axes[cluster_num]
        cluster_data = long_df[long_df['Cluster'] == cluster_num]
        sns.scatterplot(data=cluster_data, x='Variable', y='Value', ax=ax, hue='abbrevs', 
                        style='abbrevs', markers=marker_dict, legend='brief')
        # Use region abbreviations
        region_abbrevs = cluster_data['abbrevs'].unique()
        ax.set_title(f'Cluster {cluster_num}: {", ".join(region_abbrevs)}')
        ax.tick_params(axis='x', rotation=45)

        # Setting the legend outside the plot
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

    # Hide any unused subplots
    for i in range(num_clusters, len(axes)):
        axes[i].set_visible(False)

    plt.tight_layout()
    plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def plot_cluster_distributions(clustered_df, ensemble_ds):
    # Convert the xarray Dataset to a pandas DataFrame
    ensemble_df = ensemble_ds.to_dataframe().reset_index()

    # Multiply 'bgws' by 100
    ensemble_df['bgws'] *= 100

    # Exclude 'Cluster' from the variables if present
    ensemble_df = ensemble_df.drop(columns=['Cluster'], errors='ignore')

    # Melt the DataFrame to long-form for plotting
    long_df = ensemble_df.melt(id_vars=['region', 'abbrevs', 'names'],
                               var_name='Variable', value_name='Value')

    # Merge with cluster assignments
    long_df = long_df.merge(clustered_df, left_on='names', right_on='Region')

    # Number of unique clusters
    num_clusters = clustered_df['Cluster'].nunique()

    # Calculate the layout for subplots (3 columns)
    num_cols = 3
    num_rows = np.ceil(num_clusters / num_cols).astype(int)

    # Create a figure with subplots
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 5 * num_rows), squeeze=False)
    axes = axes.flatten()

    # Define markers for regions
    markers = ['o', 's', 'D', '^', 'v', '<', '>', 'p', '*', 'h', 'H', 'X']  # All filled markers
    unique_abbrevs = long_df['abbrevs'].unique()
    marker_dict = {abbrev: markers[i % len(markers)] for i, abbrev in enumerate(unique_abbrevs)}

    # Iterate over each cluster to create a scatter plot
    for cluster_num in range(num_clusters):
        ax = axes[cluster_num]
        cluster_data = long_df[long_df['Cluster'] == cluster_num]
        sns.scatterplot(data=cluster_data, x='Variable', y='Value', ax=ax, hue='abbrevs', 
                        style='abbrevs', markers=marker_dict, legend='brief')
        # Use region abbreviations
        region_abbrevs = cluster_data['abbrevs'].unique()
        ax.set_title(f'Cluster {cluster_num}: {", ".join(region_abbrevs)}')
        ax.tick_params(axis='x', rotation=45)

        # Setting the legend outside the plot
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

    # Hide any unused subplots
    for i in range(num_clusters, len(axes)):
        axes[i].set_visible(False)

    plt.tight_layout()
    plt.show()

In [ ]:
plot_cluster_distributions(clustered_df, ds)

In [ ]:
# drop regions that cluster alone:
RAR

In [ ]:
# 